In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/insurance/insurance.csv


In [18]:
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv("/kaggle/input/insurance/insurance.csv")
df = df.drop_duplicates()
df['sex'] = pd.factorize(df['sex'])[0] + 1
df['region'] = pd.factorize(df['region'])[0] + 1
df['smoker'] = pd.factorize(df['smoker'])[0] + 1
corr = df.corr()
corr['charges'].sort_values(ascending=False)

charges     1.000000
age         0.298308
bmi         0.198401
children    0.067389
sex         0.058044
region      0.006547
smoker     -0.787234
Name: charges, dtype: float64

# ML model




In [36]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import cross_val_score

In [26]:
X = df.drop('charges', axis = 1)
y = df['charges']

X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.3, random_state=101)


In [27]:
# применим стандартизацию
scaler= StandardScaler()
scaler.fit(X_train)
X_train_scaled= scaler.transform(X_train)
X_test_scaled= scaler.transform(X_test)
X_train_scaled


array([[ 0.91437827, -1.00321372,  1.25713624,  0.75514165,  0.53488998,
         1.41354968],
       [-0.22687811,  0.99679657,  0.14285056, -0.92394858,  0.53488998,
        -1.35144813],
       [ 0.84304975, -1.00321372,  1.43853158, -0.08440347,  0.53488998,
        -1.35144813],
       ...,
       [ 1.27102089,  0.99679657,  0.54937048, -0.92394858,  0.53488998,
         0.49188375],
       [-1.51079153, -1.00321372,  1.38346514, -0.92394858,  0.53488998,
        -0.42978219],
       [ 0.84304975,  0.99679657, -0.84510621, -0.92394858,  0.53488998,
        -1.35144813]])

In [37]:
# введем ошибку
def scores(y, predictions):
    mae = mean_absolute_error(y, predictions)
    mse = mean_squared_error(y, predictions)
    rmse = np.sqrt(mean_squared_error(y, predictions))
    r_squared = r2_score(y, predictions)
    return mae, mse, rmse, r_squared

models = pd.DataFrame(columns=["Model","MAE","MSE","RMSE","R2 Score"])



# Линейно-регрессионная модель

In [38]:
linear_reg_model= LinearRegression()
linear_reg_model.fit(X_train_scaled, y_train)
y_pred = linear_reg_model.predict(X_test_scaled)
y_pred = pd.DataFrame(y_pred)
MAE_li_reg= metrics.mean_absolute_error(y_test, y_pred)
MSE_li_reg = metrics.mean_squared_error(y_test, y_pred)
RMSE_li_reg =np.sqrt(MSE_li_reg)
pd.DataFrame([MAE_li_reg, MSE_li_reg, RMSE_li_reg], index=['MAE_li_reg', 'MSE_li_reg', 'RMSE_li_reg'], columns=['Metrics'])

,Metrics
MAE_li_reg,3.995758e+03
MSE_li_reg,3.511320e+07
RMSE_li_reg,5.925639e+03


In [39]:
scores = cross_val_score(linear_reg_model, X_train_scaled, y_train, cv=5)
print(np.sqrt(scores))

[0.8815739  0.84104889 0.85509915 0.86939486 0.85788515]


In [40]:
r2_score(y_test, linear_reg_model.predict(X_test_scaled))

0.7494359237216639